In [4]:
#GA con ajuste polinomial de grado 10 por cada angulo. 4x10-> 40 genes reales.
#Inicio aleatoria entre -20 y 20
from __future__ import division
from ThePablos import *
import random
from deap import base, creator, tools, algorithms
import time


#FUNCIONES AUXILIARES
def checkBoundis(individual):
    global cotainf
    global cotamax
    thetaAux = getPathFromInd(individual) - condborde.reshape((4,1))
    cotainfAux = cotainf - condborde
    cotamaxAux = cotamax - condborde
    for i in range(4):
        factor = 1
        for k in range(Npos): 
            
            if thetaAux[i,k] > cotamaxAux[i]:
                factorTemp = cotamaxAux[i]/thetaAux[i,k]
                if factorTemp < factor: factor = factorTemp
                    
            elif thetaAux[i,k] < cotainfAux[i]:
                factorTemp = cotainfAux[i]/thetaAux[i,k]
                if factorTemp < factor: factor = factorTemp 
        individual[ i*n_coef:(i*n_coef+Npos)] = individual[i*n_coef:(i*n_coef+Npos)] *factor       
    return individual
    

def kernel(X,Y,sigma):
    return  np.exp( -sum((X-Y)**2) / (2*sigma**2) )


#CONVERTIR INDIVIDUO A TRAYECTORIA
def getPathFromInd(individual):
    global Npos
    global t_mov
    global condborde
    global n_coef    
    
    t1 = np.ones(Npos)*condborde[0]
    t2 = np.ones(Npos)*condborde[1]
    t3 = np.ones(Npos)*condborde[2]
    t4 = np.ones(Npos)*condborde[3]
    theta = np.vstack([t1,t2,t3,t4])
    for i in range(4):    
        for k in range(Npos):
            for j in range(0,n_coef):
                theta[i,k] += individual[i*n_coef+j]*((k*t_mov/Npos)**(j+1))
    return theta    

#Calculo del fitness a partir del path
def evaluatePath(theta):
    global finalpos
    global robotArm
    global lam
    global en_scale
    global sigmaEnergy
    
    robotArm.setPath(theta[0], theta[1], theta[2], theta[3])
    robotArm.angularCons()
    finalGenerado = robotArm.endPoint()
    energiac = robotArm.getEnergySim()
    
    #fitness1 = mse(finalGenerado, finalpos)
    #fitness1 = 1/(1+mse(finalGenerado, finalpos))
    fitness1 = kernel(finalGenerado, finalpos,5)
    
    #fitness2 = energiac
    #fitness2 = 1/(1+energiac)
    fitness2 = kernel(np.array([energiac]), np.zeros(1),sigmaEnergy)
    
    
    fitness = (1-lam)*fitness1 + lam*fitness2
    
    return fitness,


#EVALUAR INDIVIDUO
def evaluate(individual):
    theta = getPathFromInd(individual)
    return evaluatePath(theta)

def kernel(X,Y,sigma):
    k =  np.exp(-sum((X-Y)**2) / (2*(sigma**2)))
    return k

def mse(x,y):
    return sum(abs((x-y)**2))




####---------------------- PARAMETROS ---------------------- #####

#Arreglos para iterar
lamArray = [0.5, 0.1, 0.01, 0.001]
#PmcArray = [0.4, 0.3, 0.2, 0.1]
sigmaEnergyArray = [50, 100, 150, 200]

error = np.zeros(5)
energia = np.zeros(5)

n_coef = 10 #ind_size = 4*n_coef, grado 10

#Probabilidades de recombinacion y mutacion
Pci = 0.9 #cada individuo
Pcc = 0.9 #cada atributo
Pmi = 0.5
Pmc = 0.1 #comentar para sin correntropia

cotainf = np.array([-180,-90,-90,-90])*np.pi/180
cotamax = np.array([180,90,90,90])*np.pi/180
l1 = 1
l2 = 1
l3 = 1
l4 = 1
r = 1e-2
#Posiciones en que se discretizan los caminos
Npos = 20
#Numero de generaciones
Ng = 70
#Numero de individuos en la poblacion
Npop = 300
#Duracion del movimiento
t_mov = 1

#configuracion inicial de angulos
condborde=np.array([90, -30, 30, -20])*np.pi/180

#Posicion target
#x,y,z plano cartesiano
#finalpos=np.array([1.93, 1.93, 1.0])
finalpos = np.array([1.0, 1.0, 2.0])
#finalpos=np.array([0, 0, 0])

#Crossover y MUt
eta_cx = 2 #SimulatedBinary
alpha_cx = 0.5 #Blend BLX-0.5
sigmaMut = 0.1

#Simulador de robot
robotArm = ThePablos(l1, l2, l3, l4, r, t_mov)

#--------------------ITERACION--------------#


for a in range(len(lamArray)):
    #for b in range(len(PmcArray)):
    for b in range(len(sigmaEnergyArray)):
        
        #Para promediar casos
        for c in range(4):

            #Parametros de esta iteracion
            lam = lamArray[a]        
            #Pmc = PmcArray[b]
            sigmaEnergy = sigmaEnergyArray[b]
            
            #Registro de fitness
            creator.create("FitnessMulti", base.Fitness, weights=(1.0,)) #maximizar
            #creator.create("FitnessMulti", base.Fitness, weights=(-1.0,)) #minimizar
            creator.create("Individual", np.ndarray, fitness=creator.FitnessMulti)

            #Asignacion de funciones para crear individuos
            toolbox = base.Toolbox()
            toolbox.register("attr_real", random.uniform, 0, 1)
            toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_real, n=4*n_coef)
            #Un individuo es un arreglo de coeficientes

            #Asignacion de funciones para crear la poblacion
            toolbox.register("population", tools.initRepeat, list, toolbox.individual)    

            #Asignacion de operadores    
            toolbox.register("evaluate", evaluate)
            #toolbox.register("mate", tools.cxBlend, alpha=alpha_cx)
            toolbox.register("mate", tools.cxSimulatedBinary, eta=eta_cx)
            toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=sigmaMut, indpb=Pmc)
            toolbox.register("select", tools.selTournament, tournsize=3)
            toolbox.register("checki", checkBoundis)

            ####---------------------- ALGORITMO GENETICO ----------------------- #####


            def main():
                pop = toolbox.population(n=Npop)
                for ind in pop:
                    toolbox.checki(ind)
                print("Start of evolution %i" % c)
                 # Evaluate the entire population
                fitnesses = list(map(toolbox.evaluate, pop))
                for ind, fit in zip(pop, fitnesses):
                    ind.fitness.values = fit

                # Begin the evolution
                for g in range(Ng):

                    # Select the next generation individuals
                    offspring = toolbox.select(pop, len(pop))
                    # Clone the selected individuals
                    offspring = list(map(toolbox.clone, offspring))
                    besties = tools.selBest(pop, 1)
                    best1 = besties[0]
                    # Apply crossover and mutation on the offspring
                    for child1, child2 in zip(offspring[::2], offspring[1::2]):
                        # cross two individuals with probability CXPB
                        if random.random() < Pci:
                            toolbox.mate(child1, child2)
                            # fitness values of the children
                            # must be recalculated later
                            toolbox.checki(child1)
                            toolbox.checki(child1)
                            del child1.fitness.values
                            del child2.fitness.values

                    for mutant in offspring:
                        #    mutate an individual with probability MUTPB
                        if random.random() < Pmi:
                            toolbox.mutate(mutant)
                            toolbox.checki(mutant)
                            del mutant.fitness.values

                    # Evaluate the individuals with an invalid fitness
                    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
                    fitnesses = map(toolbox.evaluate, invalid_ind)
                    for ind, fit in zip(invalid_ind, fitnesses):
                        ind.fitness.values = fit 

                    # The population is entirely replaced by the offspring
                    pop[:] = offspring
                    indaux = np.random.randint(0, len(pop))
                    pop[indaux] = best1 #Elitismo

                best_ind = tools.selBest(pop, 1)[0]
                return best_ind

            ####---------------------- VISUALIZACION DE RESULTADOS ----------------------- #####    


            if __name__ == "__main__":
                #best, max1, max2, min1, min2, prom1, prom2 = main()
                best = main()
                tiempo = np.linspace(0.0,t_mov,num=Npos)
                thetaWin = getPathFromInd(best)
                thetaWinA = thetaWin[0]
                thetaWinB = thetaWin[1]
                thetaWinC = thetaWin[2]
                thetaWinD = thetaWin[3]
                robotArm.setPath(thetaWinA, thetaWinB, thetaWinC, thetaWinD)
                robotArm.angularCons()
                finalGenerado = robotArm.endPoint()
                error[c] = mse(finalGenerado,finalpos)
                energia[c] = robotArm.getEnergySim()
        
        promedioError = sum(error)/len(error)
        stdError = np.std(error)
        promedioEnergia = sum(energia)/len(energia)
        stdEnergia = np.std(energia)
        
        file = open("TablaPoliCorr.txt", "a")
        linea = "lam "+str(lam)+" sigmaEnergy "+str(sigmaEnergy)+" error "+str(promedioError)+"std"+str(stdError)+" energia "+str(promedioEnergia)+"std"+str(stdEnergia)+" \n"
        file.write(linea)
        file.close()
        
        
        

Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evolution 3
Start of evolution 0
Start of evolution 1
Start of evolution 2
Start of evol

In [ ]:
a = 2e-3
print a
file = open("newfile.txt", "a")

file.write(str(a)+" "+str(a)+"\n")

file.write("And here is another line\n")

file.close()